In [0]:
#Colab Notebook: https://colab.research.google.com/drive/16a5d65geNrzRAlFR6xcUv3QfXV__Qyco#scrollTo=_E_gmbLIEcYc
#GitHub Repo: https://github.com/mansueto-institute/adaptive-lockdown/tree/master/studies/bihar-colab
#GDrive GitHub folder: https://drive.google.com/drive/folders/1jfJFMOP4LuSAaJo1f_bTR6RzACczVLbE
#GDrive Data folder: https://drive.google.com/drive/folders/1HZwN1zO7LZrjF8ip8czdzmlA1VqLOcUg

%%capture
!pip install --upgrade statsmodels 
!pip install geopandas

In [0]:
import csv
import numpy as np
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt 
from pathlib import Path
from typing import Dict, Optional, Sequence, Tuple
import statsmodels as sm
from datetime import datetime
import pandas as pd
from os.path import join  
from google.colab import drive 
drive.mount("/content/drive", force_remount=True)  # Mount the Google Drive at /content/drive

%matplotlib inline

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
## USER INPUT CELL
# Data folder on Google Drive
DATA_PATH = '/content/drive/My Drive/COVID-India/Districts' 
!mkdir -p "{DATA_PATH}"
# Generate a read-access token https://github.com/settings/tokens
GIT_TOKEN = '' 


In [0]:
# Root and repo path
ROOT_PATH = "/content/drive/My Drive/"
REPO_PATH = "/GitHub/" 

# Set up Git credentials
GIT_ORG = "mansueto-institute" 
GIT_REPOSITORY = "adaptive-lockdown" 


In [0]:
# Clone repo to My Drive
GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_ORG + "/" + GIT_REPOSITORY + ".git"
!mkdir -p "{ROOT_PATH}{REPO_PATH}"
%cd "{ROOT_PATH}{REPO_PATH}" 
!git clone "{GIT_PATH}" 


/content/drive/My Drive/GitHub
fatal: destination path 'adaptive-lockdown' already exists and is not an empty directory.


In [0]:
# Update repo if it already exists
#%%capture
%cd "{ROOT_PATH}{REPO_PATH}{GIT_REPOSITORY}"
!git pull 
%ls


/content/drive/My Drive/GitHub/adaptive-lockdown
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 24 (delta 10), reused 16 (delta 10), pack-reused 8
Unpacking objects: 100% (24/24), done.
From https://github.com/mansueto-institute/adaptive-lockdown
 * [new branch]      india-districts-update -> origin/india-districts-update
   5d2b44e..eac3439  studies/maharashtra-set-up -> origin/studies/maharashtra-set-up
Already up to date.
adaptive/           docs/    README.md         setup.py  studies/
adaptive-lockdown/  LICENSE  requirements.txt  src/      toy_example.py


In [0]:
# Install repo into Colab notebook 
%%capture
!pip install . 


In [0]:
# Import modules from repo
from adaptive.estimators import rollingOLS 
from adaptive.model import Model, ModelUnit
from adaptive.model import MigrationSpikeModel, Model, ModelUnit
from adaptive.plots import gantt_chart, plot_simulation_range
from adaptive.policy import simulate_adaptive_control, simulate_lockdown
from adaptive.policy import AUC, simulate_adaptive_control, simulate_lockdown
from adaptive.utils import cwd, days, fmt_params, weeks


<Figure size 432x288 with 0 Axes>

In [0]:
## USER INPUT CELL
# Set to study folder in GitHub repo containing custom functions
%cd "{ROOT_PATH}{REPO_PATH}{GIT_REPOSITORY}/studies/india_districts" 
%ls
# Import modules from study
import etl
import lockdown_evaluation


/content/drive/My Drive/GitHub/adaptive-lockdown/studies/india_districts
etl.py  lockdown_evaluation.py  raw_data1.csv  raw_data3.csv
figs/   __pycache__/            raw_data2.csv  raw_data4.csv


In [0]:
%cd "{DATA_PATH}" 

/content/drive/My Drive/COVID-India/Districts


In [0]:
# model details 
gamma      = 0.2
prevalence = 1
total_time = 90 * days 
release_date = pd.to_datetime("May 31, 2020")
lockdown_period = (release_date - pd.to_datetime("today")).days

states = ["Telangana", "Maharashtra", "Andhra Pradesh", "Tamil Nadu", "Madhya Pradesh", "Punjab", "Gujarat", "Kerala"][:4]

# use gravity matrix for states after 2001 census 
new_state_data_paths = { 
    "Telangana": ("/content/drive/My Drive/COVID-India/telangana.json", "/content/drive/My Drive/COVID-India/telangana_pop.csv")
}

# define data versions for api files
paths = { "v3": ["raw_data1.csv", "raw_data2.csv"], "v4": ["raw_data3.csv", "raw_data4.csv"] } 

# download data from india covid 19 api
for f in paths['v3'] + paths['v4']:
  etl.download_data_gdrive(DATA_PATH, f)

# run rolling regressions on historical national case data 
dfn = etl.load_all_data(
    v3_paths = paths['v3'], 
    v4_paths = paths['v4']
)
data_recency = str(dfn["date_announced"].max()).split()[0]
tsn = etl.get_time_series(dfn)
grn = adaptive.estimators.rollingOLS(tsn, window = 5, infectious_period = 1/gamma)

# disaggregate down to states
dfs = {state: dfn[dfn["detected_state"] == state] for state in states}
tss = {state: etl.get_time_series(cases) for (state, cases) in dfs.items()}
for (_, ts) in tss.items():
    ts['Hospitalized'] *= prevalence

grs = {state: adaptive.estimators.rollingOLS(timeseries, window = 5, infectious_period = 1/gamma) for (state, timeseries) in tss.items() if len(timeseries) > 5}

# voluntary and mandatory reproductive numbers
Rvn = np.mean(grn["2020-03-24":"2020-03-31"].R)
Rmn = np.mean(grn["2020-04-01":].R)
Rvs = {s: np.mean(grs[s]["2020-03-24":"2020-03-31"].R) if s in grs else Rvn for s in states}
Rms = {s: np.mean(grs[s]["2020-04-01":].R)             if s in grs else Rmn for s in states}

# voluntary and mandatory distancing rates 
Bvs = {s: R * gamma for (s, R) in Rvs.items()}
Bms = {s: R * gamma for (s, R) in Rms.items()}

migration_matrices = etl.district_migration_matrices("/content/drive/My Drive/COVID-India/Migration Matrix - District.csv", states = states)

# seed range 
si, sf = 0, 1000

for state in states: 
    if state in new_state_data_paths.keys():
        districts, populations, migrations = adaptive.model.gravity_matrix(*new_state_data_paths[state])
    else: 
        districts, populations, migrations = migration_matrices[state]
    df_state = dfs[state]
    dfd = {district: df_state[df_state["detected_district"] == district] for district in districts}
    tsd = {district: etl.get_time_series(cases) for (district, cases) in dfd.items()}
    for (_, ts) in tsd.items():
        if 'Hospitalized' in ts:
            ts['Hospitalized'] *= prevalence
    grd = {district: adaptive.estimators.rollingOLS(timeseries, window = 5, infectious_period = 1/gamma) for (district, timeseries) in tsd.items() if len(timeseries) > 5}

    Rv = {district: np.mean(grd[district]["2020-03-24":"2020-03-31"].R) if district in grd.keys() else Rvs[state] for district in districts}
    Rm = {district: np.mean(grd[district]["2020-04-01":].R)             if district in grd.keys() else Rms[state] for district in districts}

    # # fil in missing values 
    for mapping, default in ((Rv, Rvs[state]), (Rm, Rms[state])):
        for key in mapping:
            if np.isnan(mapping[key]):
                mapping[key] = default
 
 